# 1.Importing packages

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint #callback can be used for early stopping and model saving
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input 

import warnings
warnings.filterwarnings('ignore')

# 2. Load the pretrained network

In [3]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False) #inlcude_top??

58889256/58889256 [==============================] - 271s 5us/step


In [4]:
model_vgg16_conv.summary() 
#14,714,688 We solved in class (The time pass example)

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

# 3. Freeze the layers

In [5]:
#non trainable params will not be 0 only when you freeze layers
for layer in model_vgg16_conv.layers:
    layer.trainable = False

# 4.Training parameters

In [6]:
img_width, img_height = 150, 150 #std image size for VGGNET: 227x227 ITs too big
train_data_dir = 'data/train'
val_data_dir = 'data/validation'
model_weights_file = 'vgg16-xfer-weights.h5' #.h5 to store weights and biases of model
nb_train_samples = 4
nb_val_samples = 4
nb_epochs = 5

# 5.Build a classification model on top of Base Network

In [7]:
input = Input(shape=(img_width, img_height, 3)) #3 cuz its color
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=[input], outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# 6.Dataset Preparation

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=4,class_mode='categorical')

Found 4 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


# 7.Training

In [9]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, steps_per_epoch=nb_train_samples, 
                              epochs=nb_epochs, validation_data=validation_generator, validation_steps=nb_val_samples)

print('Training Completed!')


Epoch 1/5
4/4 [==============================] - 6s 1s/step - loss: 0.7095 - accuracy: 0.5000 - val_loss: 4.2917 - val_accuracy: 0.5000
Training Completed!


# 8.Test the model

In [10]:
img_path = 'data/dog.jpg'
label = ['Cat','Dog']
img = load_img(img_path, target_size=(150, 150))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features == 1)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

1/1 [==============================] - 0s 483ms/step


IndexError: index 0 is out of bounds for axis 0 with size 0

# Models for image classification with weights trained on ImageNet

